In [2]:
# from oficial python
import numpy as np
import matplotlib.pyplot as plt
from qutip import Bloch
from matplotlib import cm
import matplotlib as mpl
from qutip import basis, concurrence, Qobj
import time
from sklearn.model_selection import train_test_split
import statistics

# own modules:
from dll.spin_procesor_modificated import Quantum_Spin_Proces
from dll.main_fun import make_IRIS_v2, w_r_json
from dll. baseline_dru_v2 import Modelo_DRU
from dll.models import qcircuit_1_qubit_mixed
from dll.cost_functions import fidelity_cost
from dll.gate_procesor_modificated import CompositeGateProcessor

In [ ]:
def plot_concurrence(path):
  before, dict_samples_before_1_qubit = w_r_json('r', path)
  N_states_bl = len(dict_samples_before_1_qubit)
  dict_states_fidelities = []
  for i in range(N_states_bl):
      dict_bl = dict_samples_before_1_qubit[i]
      Fidelities_by_state = {}
      for key in dict_bl:
          state_bl_i = dict_bl[key].numpy()
          state_bl_i  = Qobj(state_bl_i)
          state_bl_i.dims = [[2, 2], [2, 2]]
          e = concurrence(state_bl_i)
          Fidelities_by_state[key] = e
      dict_states_fidelities.append(Fidelities_by_state)

  ket_labels = list(dict_states_fidelities[0].keys())
    #if entanglement == True and model.n_qubits == 2:
    #  ket_labels_e = []
    #  for i in range(len(dict_states_fidelities[0])):
    #    ket_labels_e.append()
  promedios = {ket: [] for ket in ket_labels}

  for diccionario in dict_states_fidelities:
      for ket, valor in diccionario.items():
          promedios[ket].append(valor)

    # Calcular el promedio para cada 'ket_1_qubit_i'
  promedios_mean = {ket: sum(valores) / len(valores) for ket, valores in promedios.items()}
  promedios_desviacion = {key: statistics.stdev(values) for key, values in promedios.items()}
  # Crear el diagrama de barras
  plt.figure(figsize=(12, 4))
  plt.grid()
  plt.bar(promedios_mean.keys(), promedios_mean.values(), yerr=list(promedios_desviacion.values()), capsize=5, color='blue', alpha=0.7)
  plt.xlabel('Qubits')
  plt.ylabel('Promedio')
  plt.title(f'Promedio de ket_{model.n_qubits}_qubits_i a través de diccionarios')
  plt.xticks(rotation=45)
  plt.ylim(min(promedios_mean.values()) - 0.1, max(promedios_mean.values()) + 0.5)
  plt.show()

In [4]:
X, y =  make_IRIS_v2(2,3)

model = Modelo_DRU(qcircuit_1_qubit_mixed, fidelity_cost,
                   epochs = 8, n_clases = 3, n_qubits = 1,
                   features = 2, alpha_noise = 0.0, save_process=True)
# splita data:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

# get parameters
paramters_1q, bias_1q = model.fit(X_train, y_train, X_test, y_test)

Epoch: 100%|██████████| 8/8 [04:52<00:00, 36.54s/it]
